<a href="https://colab.research.google.com/github/vitroid/ClustIce/blob/tilecycles/clustice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install py3Dmol

In [ ]:
pip install git+https://github.com/vitroid/ClustIce.git@tilecycles

In [ ]:
from logging import getLogger, INFO, DEBUG, basicConfig

import networkx as nx
import py3Dmol

from clustice.geometry import make_layout
from clustice.gromacs import render
from clustice.topology import ice_graph
from clustice.coder import encode
from clustice.water import tip4p
from clustice.graph import great_icosahedron


logger = getLogger()
basicConfig(level=DEBUG)

# note: g must be a graph whose labels start from 0.
# g = nx.dodecahedral_graph() # dodecahedral 20mer
g = great_icosahedron(2)

# O-O distance
L = 0.27

# estimate of the positions of the nodes
layout = make_layout(g, edgelen=L)

# set orientations of the hydrogen bonds.
# if pos is given, the net dipole moment is minimized.
dg = ice_graph(g, pos=layout)
# dg = ice_graph(g)

# get the unique id for the graph
id = encode(dg)
print(id)

# put water molecules
gro = render(dg, layout, watermodel=tip4p)
with open('save.gro','w') as f:
    f.write(gro)

# show
view = py3Dmol.view()
view.addModel(gro, 'gro')
view.setStyle({'stick':{}})
view.addUnitCell()
view.zoomTo()
view.show()


Rebuild the cluster from id.

In [ ]:
import networkx as nx

from clustice.geometry import make_layout
from clustice.gromacs import render
from clustice.coder import decode
from clustice.water import tip4p

import py3Dmol

s = "1.2.3+6.19.3.4+6.7.14.1+7.8+13.0+9.10+12.16.12.13+15.5.15+17.4+18.11.0+18"

dg = decode(s)

# Make layout of the nodes
g = nx.Graph(dg)
L = 0.27
layout = make_layout(g, edgelen=L)

# put water molecules
gro = render(dg, layout, watermodel=tip4p)

# show
view = py3Dmol.view()
view.addModel(gro, 'gro')
view.setStyle({'stick':{}})
view.addUnitCell()
view.zoomTo()
view.show()


In [ ]:
pip install GenIce2

In [2]:
import networkx as nx
from genice2.genice import GenIce
from genice2.plugin import Format, Lattice

from clustice.gromacs import render
from clustice.topology import ice_graph
from clustice.water import tip4p

import py3Dmol

lattice = Lattice("1h")
formatter = Format("raw", stage=(1, 2))
raw = GenIce(lattice, signature="Ice Ih", rep=(3, 3, 3)).generate_ice(
    formatter
)

# graph is the topology of the hydrogen-bond network
g = nx.Graph(raw["graph"])
# reppositions contains the positions of CoM of water in fractional coordinate
layout = raw["reppositions"]
# repcell is the cell matrix (transposed)
cell = raw["repcell"]

# set orientations of the hydrogen bonds.
# It automatically depolarizes by optimizing the arrangement.
dg = ice_graph(g, pos=layout, pbc=True)

# put water molecules
gro = render(
    dg,
    layout @ cell,
    watermodel=tip4p,
    cell=f"{cell[0,0]} {cell[1,1]} {cell[2,2]}",
    pbc=True,
)

# show
view = py3Dmol.view()
view.addModel(gro, 'gro')
view.setStyle({'stick':{}})
view.addUnitCell()
view.zoomTo()
view.show()


INFO:root:Cell dimension:
INFO:root:  a = 7.84813412606925
INFO:root:  b = 7.37735062301457
INFO:root:  c = 9.06573834219084
INFO:root:  A = 90.0
INFO:root:  B = 90.0
INFO:root:  C = 90.0
INFO:root:HB connectivity is not defined.
INFO:root:Bond length (specified): 3
INFO:root:Target Density: 0.92
INFO:root:Original Density: 0.0009111328713390615
INFO:root:Bond length (scaled, nm): 0.29903306849378936
INFO:root:__init__: end.
INFO:root:__init__: 10 ms
INFO:root:Stage1: Replicate water molecules to make a repeated cell.
INFO:root:  Number of water molecules: 432
INFO:root:  Pairs are not given explicitly.
INFO:root:  Estimating the bonds according to the pair distances.
INFO:root:  Number of water nodes: 16
INFO:root:Cell dimension:
INFO:root:  a = 0.7822838765564372
INFO:root:  b = 0.7353572647182051
INFO:root:  c = 0.9036518515423753
INFO:root:  A = 90.0
INFO:root:  B = 90.0
INFO:root:  C = 90.0
INFO:root:Stage1: end.
INFO:root:Stage1: 10 ms
INFO:root:Hook1: Replicate water molecules t

INFO:root:29 1.9999999999999996 dipole
INFO:root:40 1.1737195956531482e-15 dipole


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

# Benchmark tests

In [1]:
import networkx as nx

from clustice.geometry import make_layout
from clustice.gromacs import render
from clustice.topology import ice_graph
from clustice.coder import encode
from clustice.water import tip4p
from clustice.graph import great_icosahedron

import py3Dmol
from logging import getLogger, INFO

logger = getLogger()
logger.setLevel(INFO)

# O-O distance
L = 0.27

# note: g must be a graph whose labels start from 0.
# g = nx.dodecahedral_graph() # dodecahedral 20mer
g = great_icosahedron(3)
layout = make_layout(g, edgelen=L)
print(len(layout))


/Users/matto/.local/share/virtualenvs/ClustIce-TrkXxxIw/lib/python3.11/site-packages/sklearn/manifold/_mds.py:298: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
%%timeit

# estimate of the positions of the nodes
layout = make_layout(g, edgelen=L)



In [ ]:
%%timeit
dg = ice_graph(g, pos=layout)



In [ ]:
%%timeit
dg = ice_graph(g)


In [ ]:
# great_icosahedron
# size  nnode   constellation        depolarization       ice rule
benchmark=[
    [0, 20,  0.146, 0.00612, 0.000410],
    [1, 100, 0.676, 0.00865, 0.00229],
    [2, 280, 3.14,  0.0178,  0.00569],
    [3, 600, 51.4,  0.024,   0.0133]